In [1]:
import ee
import ee_utils
import pandas as pd
from typing import Any, Mapping, Optional, Tuple, Union
Numeric = Union[int, float]
import numpy as np
import os, requests
from IPython.display import Image

In [2]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWh2C3sEPWsTIBqsL8YFv7Ja9Ly4W8d1Wqi4FNhXi3jDXQIGdF9sPxM

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [35]:
def save_an_image(datacode, bands, coords, fname, start_date, end_date, dim=256, qaclear=True):
    LON, LAT= coords
    roi = ee.Geometry.Point(LON,LAT).buffer(1000) # in meters. Singapore width is 26km
    imgcol = ee.ImageCollection(datacode)
    imgcol = imgcol.filterDate(start_date,end_date).filterBounds(roi)
    imgcol = imgcol.map(ee_utils.mask_qaclear).select(MS_BANDS)
    img = imgcol.mean().select(bands)
    
    imgcol = ee.ImageCollection(datacode)
    imgcol = imgcol.filterDate(start_date,end_date).filterBounds(roi)
    # print('imgcol.size().getInfo():',imgcol.size().getInfo())
    if qaclear:
        imgcol = imgcol.map(ee_utils.mask_qaclear)
    imgcol = imgcol.select(bands)
    img = imgcol.median() #  mean() also work
    
    url = img.getThumbUrl({'min':0,'max':65455,'dimensions': dim, 'region': roi,}) # max 65455 is from the website's specification
    print(url)

    page = requests.get(url)
    f_name = fname +'.png'
    with open(f_name, 'wb') as f:
        f.write(page.content)

In [37]:
LAT, LON = -12.350257, 13.534922

country, year = 'Angola', 2011
fname = '%s_%s'%(country, str(year))
MS_BANDS = ['SR_B1','SR_B2','SR_B3', 'QA_PIXEL'] # 'BLUE, GREEN, RED, Pixel quality attributes generated from the CFMASK algorithm.'

# image export parameters
start_date, end_date = ee_utils.surveyyear_to_range(year)
print(start_date, end_date)

L7CODE = 'LANDSAT/LE07/C02/T1_L2'
save_an_image(datacode=L7CODE, 
              bands=['SR_B1','SR_B2','SR_B3'], coords=(LON, LAT),
              fname=fname,start_date=start_date, end_date=end_date, dim='512')
save_an_image(datacode=L7CODE, 
              bands=['SR_B4','SR_B5','SR_B7'], coords=(LON, LAT),
              fname=fname + '_echan',start_date=start_date, end_date=end_date, dim='512')

2009-1-1 2011-12-31
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/fc762b6a7a2d03d1bee0b33d585471f9-2e715dec1234eb2bfe2e029c201863e0:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/292a144b96a4f520427b10531aecfcd0-40349642722351913980ed2492f6e8ca:getPixels


In [39]:
L8CODE = 'LANDSAT/LC08/C02/T1_L2'
l8_bands = ['SR_B2','SR_B3','SR_B4']
start_date, end_date = '2015-1-1', '2017-12-31'
fname = 'Angola_2015'
save_an_image(datacode=L8CODE, 
              bands=l8_bands, coords=(LON, LAT),
              fname=fname,start_date=start_date, end_date=end_date, dim='512')
save_an_image(datacode=L8CODE, 
              bands=['SR_B5','SR_B6','SR_B7'], coords=(LON, LAT),
              fname=fname+'_echan',start_date=start_date, end_date=end_date, dim='512')

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6c2ed9f1ea5e53a4f089a55b8f18d00c-b8bc7625b2635a980c1c0bd8b7a896e5:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/8c917b0c5f85bc181a3c7786d0c15bc1-5069a8af835730f8eb3f4a7d4e913067:getPixels
